## This laboratory will be conducted over two sessions
## You are expected to finish the software and test the code in simulation between the sessions! ##

# Manipulating objects

For this laboratory, we will use two robots distant from each other of 25cm as depicted in the figure below

<img src="./2_robot_config.png" width="500">

There is 1cm between the end-effector "shell" and the position of the frame as depicted in the figure below

<img src="./foot_close.png" width="300">

<img src="./box.jpg" width="200">

The objects to grasp are soft cubes with a 5cm side.
Three cubes are stacked on top of each other. The goal of the laboratory is to design a controller to will move the two cubes on the top away
and pick the bottom cube and lift it up by 10cm.

In [52]:
#setup nice plotting (use widget instead of notebook in the command below if you use jupyter lab)

# we import useful libraries
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt
from time import sleep, time

# this function returns the body Jacobian
from robot_utils import getBodyJacobianOrientedLikeSpatialFrame

# we import the robot class
use_real_robot = True

if use_real_robot:
    from NYUFinger.real import NYUDualFingerHardware as Robot   
else:
    from NYUFinger.sim import NYUDualFingerSimulator as Robot


# here we define the global variables for the robot size
l0 = 0.3
l1 = 0.16
l2 = 0.16
l3 = 0.014

## Instantiate The Robots
If you're using the real robots, put them into zero position and then run the next cell to instantiate and reset them. 

In [2]:
robot = Robot()

In [4]:
if use_real_robot:
    robot.reset_sensors()
else:
    robot.reset()

Successfully reset the sensor values to: [0. 0. 0.]
Successfully reset the sensor values to: [0. 0. 0.]


You can also visualize the robot to ensure that the real movement of the robot matches the 3D simulator environment (Only if you're using the real robot)

In [ ]:
import time
if use_real_robot:
    from NYUFinger.utils.vis import NYUDualFingerVisualizer
    vis = NYUDualFingerVisualizer()
    start_time = time.time()
    while time.time()-start_time < 60:
        q, dq = robot.get_state()
        robot.send_joint_torque(np.zeros(6))
        vis.show(q)
        time.sleep(0.01)
else:
    print('This visualization is meaningless for the simulated robot experiment')

This visualization is meaningless for the simulated robot experiment


## Interacting with the Robot
For this lab, you'll be using two robots at the same time. Unlike the previous labs where the $\tau$, $q$, and $dq$ were 3D vectors, in this lab robot takes 6D commands and returns 6D states. The first three numbers are for finger 1 (left) and the last three are for the finger 2 (right). Let's make sure you can read the sates and command the robot:

In [ ]:
q, dq = robot.get_state()
q1 = q[:3]
q2 = q[3:]
dq1 = dq[:3]
dq2 = dq[3:]
print(f'Robot1: q={q1}, dq1={dq1}')
print(f'Robot2: q={q2}, dq1={dq2}')

Robot1: q=[-1.82528376e-03 -1.91072789e-19  4.09331595e-19], dq1=[-2.27623448e-01 -3.27377994e-17  6.93458024e-17]
Robot2: q=[-1.82528376e-03 -2.25163247e-19  2.12374375e-19], dq1=[-2.27623448e-01 -4.43029968e-17  5.37799173e-17]


If you're using the simulator, you can also access the ground truth posiion of the cubes using `robot.get_cube_positions()` and reset their location using `robot.reset()`: 

In [ ]:
if not use_real_robot:
    robot.reset()
    print(robot.get_cube_positions())

{'cube1': array([ 0.125     , -0.05      ,  0.02498212]), 'cube2': array([ 0.125     , -0.05      ,  0.07497303]), 'cube3': array([ 0.125     , -0.05      ,  0.12496797])}


## Questions
In this laboratory you are expected to re-use code from the previous laboratories (e.g. to plot, compute forward kinematics, etc) and solve the problem in both simulation and using the real robot. For both simulation and real robot deployment, you're expected to upload a short video of the robot solving the task alongside the report containing plots showing the robot reaching the desired 10cm goal for the designated cube.

1. Using an impedance controller (Lab4 - use the ``getBodyJacobianOrientedLikeSpatialFrame`` function to get the Jacobian) and one trajectory generation method seen in Lab3 write a complete controller that solves the problem: move the top two cubes away, grasp the last cube and move it up by 10cm. In the report, describe in details the controller you designed (what it does, etc) and the results you obtained.

2. Test your controller in simulation. Save and plot the results.
   
3. Test the controller on the real robots. Save and plot the results.

4. (optional - bonus) Extend the controller to rearrange the cubes - putting the bottom one on top of the two others.

In [37]:
### COPY PASTE YOUR FORWARD KINEMATICS FUNCTION HERE FROM LAB 2 (and other helper functions)
def forward_kinematics_1(q):
    ###WRITE YOUR CODE HERE
    T_SH1 = translate([l0,0,0])
    
    T_SH2 = rotateX(q[0]) @ translate([0,0,l3])
    
    T_SK = rotateZ(q[1]) @ translate([0,-l1,0])
    
    T_SF = rotateZ(q[2]) @ translate([0,-l2,0])
    

    FK = T_SH1 @ T_SH2 @ T_SK @ T_SF
    ### we return the object
    return FK

def translate(vector):
    # we allocate a 4x4 array (as identity since this corresponds to no motion)
    transform = np.eye(4)
    
    # here you can fill the rest of the transform
    ##WRITE CODE HERE
    transform[0,3] = vector[0]
    transform[1,3] = vector[1]
    transform[2,3] = vector[2]

    ### we return the object
    return transform

def rotateX(angle):
    # we allocate a 4x4 array (as identity since this corresponds to no motion)
    transform = np.eye(4)
    
    # here you can fill the rest of the transform
    ##WRITE CODE HERE
    transform[1,1] = np.cos(angle)
    transform[1,2] = -np.sin(angle)
    transform[2,1] = np.sin(angle)
    transform[2,2] = np.cos(angle)
    
    ### we return the object
    return transform

def rotateY(angle):
    # we allocate a 4x4 array (as identity since this corresponds to no motion)
    transform = np.eye(4)
    
    # here you can fill the rest of the transform
    ##WRITE CODE HERE
    transform[0,0] = np.cos(angle)
    transform[0,2] = np.sin(angle)
    transform[2,0] = -np.sin(angle)
    transform[2,2] = np.cos(angle)
    
    ### we return the object
    return transform

def rotateZ(angle):
    # we allocate a 4x4 array (as identity since this corresponds to no motion)
    transform = np.eye(4)
    
    # here you can fill the rest of the transform
    ##WRITE CODE HERE
    transform[0,0] = np.cos(angle)
    transform[0,1] = -np.sin(angle)
    transform[1,0] = np.sin(angle)
    transform[1,1] = np.cos(angle)
    
    ### we return the object
    return transform

In [38]:
def forward_kinematics_2(q):
    T = forward_kinematics_1(q)
    T[0, 3] -= 0.25
    return T


In [20]:
def compute_trajectory(position_init, position_goal, t_init, t_goal, t):
    T = t_goal - t_init
    desired_position = position_init + ((3/T**2)*(t-t_init)**2 - (2/T**3)*(t-t_init)**3)*(position_goal - position_init)    
    desired_velocity = ((6/T**2)*(t-t_init)-(6/T**3)*(t-t_init)**2)*(position_goal-position_init)
    
    # we return the answer
    return desired_position, desired_velocity

In [ ]:
q = robot.get_state()[0]
pos1 = forward_kinematics_1(q[:3])[:3, 3:]
pos2 = forward_kinematics_2(q[3:])[:3, 3:]
averge = ((pos1+pos2)/2).squeeze()
print(list(averge))

[[ 0.21119493]
 [-0.17234563]
 [ 0.01062747]]
[[ 0.20494033]
 [-0.14854797]
 [ 0.01819728]]
[np.float64(0.20806763056758493), np.float64(-0.1604468002627711), np.float64(0.014412377448966018)]


In [ ]:
cube_positions = {"cube3": np.array([np.float64(0.20), np.float64(-0.16), np.float64(0.01)])}
if not use_real_robot:
    cube_positions = robot.get_cube_positions()

# we simulate for 10 seconds
run_time = 5
dt = 0.001
num_steps = int(run_time/dt)

# the PD gains
P = np.array([100, 100, 100])
D = np.array([1, 1, 1])


## we allocate a few vectors to collect data while we do the simulation
# we keep track of time
time0 = np.zeros([num_steps])
# we will store the sensor values (angular position and velocities)

measured_positions_f1 = np.zeros([num_steps,3])
measured_velocities_f1 = np.zeros_like(measured_positions_f1)

measured_positions_f2 = np.zeros_like(measured_positions_f1)
measured_velocities_f2 = np.zeros_like(measured_positions_f1)

# we will store the end effector position (desired and measured)

measured_finger_position_f1 = np.zeros([num_steps,3])
desired_finger_position_f1 = np.zeros([num_steps,3])

measured_finger_position_f2 = np.zeros([num_steps,3])
desired_finger_position_f2 = np.zeros([num_steps,3])

q0, dq0 = robot.get_state()
q0_1 = q0[:3]
q0_2 = q0[3:]
q_prev_1 = q0_1
q_prev_2 = q0_2
t_prev = 0

t_goals = [5]

for i in range(num_steps):
    # get the current time and save it
    t_start = time()
    time0[i] = dt * i
    
    # we get the position and velocities of the joints
    q, dq = robot.get_state()
    q1 = q[:3]
    q2 = q[3:]
    dq1 = dq[:3]
    dq2 = dq[3:]

    pose_1 = forward_kinematics_1(q1)
    measured_finger_position_f1[i,:] = pose_1[0:3,3]
    p_measured_1 = measured_finger_position_f1[i]

    pose_2 = forward_kinematics_2(q2)
    measured_finger_position_f2[i,:] = pose_2[0:3,3]
    p_measured_2 = measured_finger_position_f2[i]
    
    # controller: TODO HERE IMPLEMENT YOUR CONTROLLER (robot 1)

    J1 = getBodyJacobianOrientedLikeSpatialFrame(q1)
    p_ref_1, v_ref_1 = compute_trajectory(q_prev_1, cube_positions["cube3"], t_prev, t_goals[0], time0[i])
    p_diff_1 = p_ref_1 - p_measured_1

    p_measured_1_dot = J1 @ dq1
    d_diff_1 = v_ref_1 - p_measured_1_dot

    measured_velocities_f1[i,:] = p_measured_1_dot

     # controller: TODO HERE IMPLEMENT YOUR CONTROLLER (robot 2)

    J2 = getBodyJacobianOrientedLikeSpatialFrame(q2)
    p_ref_2, v_ref_2 = compute_trajectory(q_prev_2, cube_positions["cube3"], t_prev, t_goals[0], time0[i])
    p_diff_2 = p_ref_2 - p_measured_2

    p_measured_2_dot = J2 @ dq2
    d_diff_2 = v_ref_2 - p_measured_2_dot

    measured_velocities_f2[i,:] = p_measured_2_dot

    #joint torques

    joint_torques1 = J1.T @ ((np.diag(P) @ p_diff_1) + (np.diag(D) @ d_diff_1))
    joint_torques1 = np.expand_dims(joint_torques1, axis=1)

    joint_torques2 = J2.T @ ((np.diag(P) @ p_diff_2) + (np.diag(D) @ d_diff_2))
    joint_torques2 = np.expand_dims(joint_torques2, axis=1)


    joint_torques = np.vstack([joint_torques1, joint_torques2]).squeeze()

    # we send them to the robot and do one simulation step
    robot.send_joint_torque(joint_torques)
    sleep(max(dt*(i+1) - (time()-t_start), 0))

NameError: name 't' is not defined

In [ ]:
#robot 1 and robot 2
def plot_joint_posvel(time0, th, th_des, dth, dth_des):
    fig = plt.figure(figsize=(10,10))
    
    plt.subplot(3,2,1)
    plt.plot(time0, th[:,0], 'b-', time0, th_des[:,0], '--k')
    plt.ylabel(r'$\theta_0$')
    plt.subplot(3,2,3)
    plt.plot(time0, th[:,1], 'b-', time0, th_des[:,1], '--k')
    plt.ylabel(r'$\theta_1$')
    plt.subplot(3,2,5)
    plt.plot(time0, th[:,2], 'b-', time0, th_des[:,2], '--k')
    plt.ylabel(r'$\theta_2$')
    plt.xlabel('Time [s]')
    
    plt.subplot(3,2,2)
    plt.plot(time0, dth[:,0], 'b-', time0, dth_des[:,0], '--k')
    plt.ylabel(r'$\dot{\theta}_0$')
    plt.subplot(3,2,4)
    plt.plot(time0, dth[:,1], 'b-', time0, dth_des[:,1], '--k')
    plt.ylabel(r'$\dot{\theta}_1$')
    plt.subplot(3,2,6)
    plt.plot(time0, dth[:,2], 'b-', time0, dth_des[:,2], '--k')
    plt.ylabel(r'$\dot{\theta}_2$')
    plt.xlabel('Time [s]')
    
plot_joint_posvel(time0, measured_positions_f1, desired_finger_position_f1, measured_velocities_f1)
plot_joint_posvel(time0, measured_positions_f2, desired_finger_position_f2, measured_velocities_f2)